# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from sklearn.model_selection import train_test_split
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core.webservice import LocalWebservice
from azureml.core.webservice import Webservice, AciWebservice
from azureml.core.model import InferenceConfig
from azureml.automl.core.shared import constants
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
import numpy as np
import pandas as pd
import requests
import joblib
import json
import logging

## Dataset


### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

## Dataset

The Wine Quality Datasets have been taken from the UCI Machine Learning Repository. The data is broken up into two individual datasets, one for red wines and the other for white wines. The red wine dataset contains 1599 examples while the white wine dataset has 4898 examples. Both datserts have the same 12 attributes as follows:

Attribute information:

For more information, read [Cortez et al., 2009].

Input variables (based on physicochemical tests):
1 - fixed acidity
2 - volatile acidity
3 - citric acid
4 - residual sugar
5 - chlorides
6 - free sulfur dioxide
7 - total sulfur dioxide
8 - density
9 - pH
10 - sulphates
11 - alcohol
Output variable (based on sensory data): 
12 - quality (score between 0 and 10)

There are no missing values. I will be combining these datasets into one dataset and using AutoML to predict if a wine is red or white.

This dataset is public and available for research. I have included the citation below as requested:

P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
Modeling wine preferences by data mining from physicochemical properties.
In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.

Available at: [@Elsevier] http://dx.doi.org/10.1016/j.dss.2009.05.016
            [Pre-press (pdf)] http://www3.dsi.uminho.pt/pcortez/winequality09.pdf
            [bib] http://www3.dsi.uminho.pt/pcortez/dss09.bib
            


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Wine_Evaluation_AutoMl'

experiment=Experiment(ws, experiment_name)

In [3]:
cluster_name = "automl-capstone-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

vm_size = "Standard_D2_V2"
max_nodes = 4

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, using it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=max_nodes)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
dataset_name = 'winequality'

# Check Registry
if dataset_name in ws.datasets.keys(): 
    dataset = ws.datasets[dataset_name] 

    print(f'Located {dataset_name} in Registry.')

# Read from UCI ML Repository
else:
# Read data
    red = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
    white = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')

# Set target classes (0: White Wine, 1: Red Wine)
    white['y'] = 0
    red['y'] = 1

# Combine into a single DataFrame & shuffle
    wine_df = pd.concat([red, white], axis=0)
    wine_df = wine_df.sample(frac=1)

# Upsample to balance dataset (Number of Instances: red wine - 1599; white wine - 4898)
    def upsample_classes(data, target):
        
        lst = list(data[target].unique())
        
        classes = []
        for c in lst:
            classes.append(data[data[target]==c])
        
        length = 0
        class_lab = None
        for c in classes:
            if len(c)>length:
                length=len(c)
                class_lab = c
        class_lab = class_lab[target].unique()[0]
        
        regroup = pd.concat(classes)
        maj_class = regroup[regroup[target]==class_lab]

        lst.remove(class_lab)
        
        new_classes=[]
        for i in lst:
            new_classes.append(resample(data[data[target]==i],replace=True, n_samples=len(maj_class)))

        minority_classes = pd.concat(new_classes)
        upsample = pd.concat([regroup[regroup[target]==class_lab],minority_classes])

        return upsample

    wine_df_balanced = upsample_classes(wine_df, 'y')


# Register the dataset
    datastore = ws.get_default_datastore()

    dataset = Dataset.Tabular.register_pandas_dataframe(
        dataframe=wine_df_balanced, 
        name=dataset_name, 
        description='A dataset of white and red wines',
        target=datastore
    )

    print(f'Read {dataset_name} from UCI ML Repository and registered dataset.')


Located winequality in Registry.


In [5]:
# Check out the DataFrame
df = dataset.to_pandas_dataframe()
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,y
0,6.6,0.210,0.30,9.9,0.041,64.0,174.0,0.99500,3.07,0.50,10.1,6,0
1,7.1,0.170,0.43,1.3,0.023,33.0,132.0,0.99067,3.11,0.56,11.7,6,0
2,7.3,0.655,0.20,10.2,0.071,28.0,212.0,0.99710,2.96,0.58,9.2,6,0
3,6.6,0.260,0.38,6.5,0.170,68.0,201.0,0.99560,3.19,0.38,9.4,6,0
4,6.8,0.290,0.16,1.4,0.038,122.5,234.5,0.99220,3.15,0.47,10.0,4,0


In [6]:

# Split the DataFrame into training and testing datasets
train, test = train_test_split(df, random_state=666, shuffle=True, test_size=0.2)
datastore = ws.get_default_datastore()
train = Dataset.Tabular.register_pandas_dataframe(
        dataframe=train, 
        name='wine_train', 
        description='A dataset of white and red wines',
        target=datastore
    )


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/4969c670-bd59-4d9e-b0f7-f2c1fd6766a4/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

I set the following AutoML parameters: 

* *experiment_timeout_minutes*: 30
* *enable_early_stopping*: True    
* *primary_metric*: 'accuracy'
* *n_cross_validations*:5
* *iterations*: 50
* *max_concurrent_iterations*: 4

The *experiment_timeout_minutes* (30 mins), *iterations* (50) and *enable_early_stopping* (True) are set to reduce time 
taken for model training. Enabling early stopping allows the training process to conclude if there is no considerable 
improvement to the *primary_metric* (accuracy). *n_cross_validations* is set to 5 to ensure Bias vs Variance tradeoff
and prevent overfitting.The *max_concurrent_iterations* (4) is set as we are running on compute Standard_D2_V2 which has 
4 nodes. This allows 4 jobs to be run in parallel on each node.


In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes":30,
    "enable_early_stopping":True,    
    "primary_metric":'accuracy',
    "n_cross_validations":5,
    "iterations":50,
    "max_concurrent_iterations":4,
    "verbosity": logging.INFO
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_target, 
    task='classification', 
    training_data=train,
    label_column_name='y',
    **automl_settings)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on automl-capstone-cluster with default configuration
Running on remote compute: automl-capstone-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
Wine_Evaluation_AutoMl,AutoML_b656a63b-cc15-48a5-80c5-57adc9bbd5e7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
status = remote_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_run, fitted_model = remote_run.get_output()


# Best Run
print("Best Run: ")
print("")

# Accuracy
print("Accuracy: " + str(best_run.get_metrics()['accuracy']))
print("")

# Metrics
print("Metrics: " + str(best_run.get_metrics()))
print("")

# Name
print("Name: " + str(best_run.properties['model_name']))
print("")

# Tags
print("Tags: " + str(best_run.get_tags()))
print("")


Package:azureml-automl-runtime, training version:1.48.0.post1, current version:1.47.0
Package:azureml-core, training version:1.48.0, current version:1.47.0
Package:azureml-dataprep, training version:4.8.3, current version:4.5.7
Package:azureml-dataprep-rslex, training version:2.15.1, current version:2.11.4
Package:azureml-dataset-runtime, training version:1.48.0, current version:1.47.0
Package:azureml-defaults, training version:1.48.0, current version:1.47.0
Package:azureml-interpret, training version:1.48.0, current version:1.47.0
Package:azureml-mlflow, training version:1.48.0, current version:1.47.0
Package:azureml-pipeline-core, training version:1.48.0, current version:1.47.0
Package:azureml-responsibleai, training version:1.48.0, current version:1.47.0
Package:azureml-telemetry, training version:1.48.0, current version:1.47.0
Package:azureml-train-automl-client, training version:1.48.0, current version:1.47.0
Package:azureml-train-automl-runtime, training version:1.48.0.post1, cur

Best Run: 

Accuracy: 0.997958044098303

Metrics: {'recall_score_macro': 0.9979588732070226, 'recall_score_micro': 0.997958044098303, 'precision_score_weighted': 0.9979583691197231, 'weighted_accuracy': 0.9979572145969062, 'AUC_macro': 0.9999431279781599, 'matthews_correlation': 0.9959141606930155, 'balanced_accuracy': 0.9979588732070226, 'norm_macro_recall': 0.9959177464140453, 'f1_score_macro': 0.9979569177036259, 'average_precision_score_micro': 0.99994497054764, 'log_loss': 0.018439937576820036, 'accuracy': 0.997958044098303, 'precision_score_macro': 0.9979552875403446, 'f1_score_weighted': 0.9979580439947883, 'AUC_weighted': 0.9999431279781599, 'average_precision_score_macro': 0.9999435795202973, 'f1_score_micro': 0.997958044098303, 'AUC_micro': 0.9999443716100597, 'precision_score_micro': 0.997958044098303, 'recall_score_weighted': 0.997958044098303, 'average_precision_score_weighted': 0.9999436689275362, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_b656a63b-cc15

In [12]:
# Locate Scoring File
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'explanation/6e51b67d/classes.interpret.json',
 'explanation/6e51b67d/eval_data_viz.interpret.json',
 'explanation/6e51b67d/expected_values.interpret.json',
 'explanation/6e51b67d/features.interpret.json',
 'explanation/6e51b67d/global_names/0.interpret.json',
 'explanation/6e51b67d/global_rank/0.interpret.json',
 'explanation/6e51b67d/global_values/0.interpret.json',
 'explanation/6e51b67d/local_importance_values.interpret.json',
 'explanation/6e51b67d/per_class_names/0.interpret.json',
 'explanation/6e51b67d/per_class_rank/0.interpret.json',
 'explanation/6e51b67d/per_class_values/0.interpret.json',
 'explanation/6e51b67d/rich_metadata.interpret.json',
 'explanation/6e51b67d/true_ys_viz.interpret.json',
 'explanation/6e51b67d/visualization_dict.interpret.json',
 'explanation/6e51b67d/ys_pred_proba_viz.interpret.json',
 'explanation/6e51b67d/ys_pred_viz.interpret.json',
 'explanation/bd2a701d/classes.interpret.json',
 'expl

In [13]:
y_test = test['y']
X_test = test.drop(['y'], axis=1)

ypred = fitted_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

,0,1
0,975,3
1,4,978


In [19]:
#TODO: Save the best model

joblib.dump(fitted_model, filename='automl_best_model.joblib')

# register model
model_name = best_run.properties['model_name']
model = remote_run.register_model(model_name=model_name)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [20]:
# Download Scoring File
best_run.download_file('outputs/scoring_file_v_1_0_0.py','score.py')

In [26]:
environment = best_run.get_environment().save_to_directory(path='environment')
entry_script= 'score.py'
inference_config = InferenceConfig(entry_script = entry_script, environment=environment)
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                    name="wine-webservice",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-01-29 00:51:18+00:00 Creating Container Registry if not exists.
2023-01-29 00:51:19+00:00 Use the existing image.
2023-01-29 00:51:20+00:00 Submitting deployment to compute.
2023-01-29 00:51:23+00:00 Checking the status of deployment wine-webservice..
2023-01-29 00:53:16+00:00 Checking the status of inference endpoint wine-webservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


## Interacting with the endpoint
TODO: In the cell below, send a request to the web service you deployed to test it.

In [27]:
url = service.scoring_uri
print(url)

test_payloads = [
    {
    'fixed acidity': 6.6,
    'volatile acidity': 0.84,
    'citric acid': 0.03,
    'residual sugar': 2.3,
    'chlorides': 0.059,
    'free sulfur dioxide': 32.0,
    'total sulfur dioxide': 48.0,
    'density': 0.9952,
    'pH': 3.52,
    'sulphates': 0.56,
    'alcohol': 12.3,
    'quality': 7
    },
    {
    'fixed acidity': 6.0,
    'volatile acidity': 0.2,
    'citric acid': 0.26,
    'residual sugar': 1.1,
    'chlorides': 0.033,
    'free sulfur dioxide': 38.0,
    'total sulfur dioxide': 67.0,
    'density': 0.98954,
    'pH': 3.14,
    'sulphates': 0.38,
    'alcohol': 11.5,
    'quality': 6
    }
]

for payload in test_payloads:

    test_payload = {"data":[payload]}
    input_data = json.dumps(test_payload)
    print(input_data)

    output = requests.post(url, input_data, headers={'Content-Type': 'application/json'})
    print(output.json())


http://de2e33ba-8d0c-43de-8824-d550f61cd983.southcentralus.azurecontainer.io/score
{"data": [{"fixed acidity": 6.6, "volatile acidity": 0.84, "citric acid": 0.03, "residual sugar": 2.3, "chlorides": 0.059, "free sulfur dioxide": 32.0, "total sulfur dioxide": 48.0, "density": 0.9952, "pH": 3.52, "sulphates": 0.56, "alcohol": 12.3, "quality": 7}]}
{"result": [1]}
{"data": [{"fixed acidity": 6.0, "volatile acidity": 0.2, "citric acid": 0.26, "residual sugar": 1.1, "chlorides": 0.033, "free sulfur dioxide": 38.0, "total sulfur dioxide": 67.0, "density": 0.98954, "pH": 3.14, "sulphates": 0.38, "alcohol": 11.5, "quality": 6}]}
{"result": [0]}


In [28]:
# Look at service logs
print(service.get_logs())

2023-01-29T00:53:01,321011800+00:00 - iot-server/run 
2023-01-29T00:53:01,323901000+00:00 - rsyslog/run 
2023-01-29T00:53:01,341813100+00:00 - gunicorn/run 
2023-01-29T00:53:01,344835900+00:00 | gunicorn/run | 
2023-01-29T00:53:01,354773200+00:00 | gunicorn/run | ###############################################
2023-01-29T00:53:01,356350300+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-01-29T00:53:01,363162700+00:00 | gunicorn/run | ###############################################
2023-01-29T00:53:01,364737900+00:00 | gunicorn/run | 
2023-01-29T00:53:01,366437000+00:00 | gunicorn/run | 
2023-01-29T00:53:01,381630400+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20230103.v4
2023-01-29T00:53:01,383985600+00:00 | gunicorn/run | 
2023-01-29T00:53:01,390085700+00:00 | gunicorn/run | 
2023-01-29T00:53:01,390464100+00:00 - nginx/run 
2023-01-29T00:53:01,391879700+00:00 | gunicorn/run | PATH environment variable: /azureml-env

TODO: In the cell below, print the logs of the web service and delete the service

In [25]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
